# Data Exploration

**Author:** Florian Klaver

In this script the structure and attributes of the datasets are explored to identify relevant layers and classification codes. This is important to define the filter criteria used in later analysis steps.

## Setup

In [7]:
import os
import geopandas as gpd
import fiona
import pandas as pd
from pathlib import Path

In [8]:
# --- PATH CONFIGURATION ---
try:
    # Try to get the script directory (works for standard .py files)
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Fallback for Jupyter Notebooks (uses current working directory)
    # Assumes the notebook is running from the 'code' directory
    script_dir = os.getcwd()

# Get project root directory (one level up from 'code')
project_root = os.path.dirname(script_dir)

# Input Directories
data_dir = os.path.join(project_root, 'data')

# File Paths
paths = {
    "boundaries": os.path.join(data_dir, "swissBOUNDARIES3D_1_5_LV95_LN02.gpkg"),
    "land_cover": os.path.join(data_dir, "SWISSTLM3D_2025.gpkg"), 
    "land_use": os.path.join(data_dir, "arealstatistik_2056.gpkg")
}

## Process


In [9]:
# --- HELPER FUNCTION ---
def inspect_gpkg_structure(path, dataset_name):
    """
    Lists all layers within a GeoPackage and inspects the attributes of relevant layers.
    """
    if not os.path.exists(path):
        print(f"❌ {dataset_name} not found at {path}")
        return

    print(f"\n" + "="*40)
    print(f"INSPECTING: {dataset_name}")
    print("="*40)
    
    try:
        # 1. List all available layers
        layers = fiona.listlayers(path)
        print(f"Total Layers: {len(layers)}")
        print(f"Layer List: {layers}")
        
        # 2. Define keywords to find relevant layers automatically
        keywords = ['STRASSE', 'ROAD', 'BODENBEDECKUNG', 'COVER', 'WALD', 'AREA', 'STANDARD']
        layers_to_check = [l for l in layers if any(k in l.upper() for k in keywords)]
        
        # Fallback: check first layer if nothing matches
        if not layers_to_check and layers:
            layers_to_check.append(layers[0])
            
        print(f"\n-> detailed inspection for: {layers_to_check}")

        # 3. Inspect attributes (columns and unique values)
        for layer_name in layers_to_check:
            print(f"\n   > Layer: '{layer_name}' (First 100 rows)...")
            
            # Load small subset for performance
            gdf = gpd.read_file(path, layer=layer_name, rows=100)
            print(f"     Columns: {list(gdf.columns)}")
            
            # Check for classification columns often used in Swiss data
            possible_class_cols = ['objektart', 'kunstbaute', 'belagsart', 'nutzung', 
                                   'AS18_4', 'AS18_17', 'AS18_72', 'Kategorie']
            
            found_cols = [c for c in possible_class_cols if c in gdf.columns]
            
            for col in found_cols:
                unique_vals = gdf[col].unique()
                print(f"     Unique values in '{col}': {unique_vals}")

    except Exception as e:
        print(f"Error inspecting {dataset_name}: {e}")

In [10]:
# --- EXECUTION ---

# 1. Inspect SwissTLM3D (Roads, Forest, Water)
inspect_gpkg_structure(paths["land_cover"], "SwissTLM3D")

# 2. Inspect Arealstatistik (Settlements, Agriculture)
inspect_gpkg_structure(paths["land_use"], "Arealstatistik")

# 3. Inspect Boundaries (Cantonal Borders)
inspect_gpkg_structure(paths["boundaries"], "SwissBOUNDARIES3D")


INSPECTING: SwissTLM3D
Total Layers: 40
Layer List: ['tlm_strassen_aus_einfahrt', 'tlm_bb_bodenbedeckung', 'tlm_bb_einzelbaum_gebuesch', 'tlm_eo_einzelobjekt', 'tlm_oev_eisenbahn', 'tlm_gewaesser_fliessgewaesser', 'tlm_namen_flurname', 'tlm_areale_freizeitareal', 'tlm_bauten_gebaeude_footprint', 'tlm_namen_gebietsname', 'tlm_name_gelaendename', 'tlm_oev_haltestelle', 'tlm_bauten_leitung', 'tlm_bauten_mauer', 'tlm_namen_name_pkt', 'tlm_areale_nutzungsareal', 'tlm_oev_schifffahrt', 'tlm_areale_schutzgebiet', 'tlm_namen_siedlungsname', 'tlm_bauten_sportbaute_lin', 'tlm_bauten_sportbaute_ply', 'tlm_bauten_staubaute', 'tlm_gewaesser_stehendes_gewaesser', 'tlm_strassen_strasse', 'tlm_strassen_strasseninfo', 'tlm_oev_uebrige_bahn', 'tlm_bauten_verbauung', 'tlm_areale_verkehrsareal', 'tlm_bauten_verkehrsbaute_lin', 'tlm_bauten_verkehrsbaute_ply', 'tlm_bauten_versorgungsbaute_pkt', 'tlm_bb_glamos', 'tlm_bauten_leitung_stromtrasse', 'tlm_areale_nutzungsareal_schule', 'tlm_areale_schule', 'tlm_s